In [3]:
# 데이터 전처리용 패키지
import pandas as pd
import numpy as np

# 패키지 쓰고 저장
import pickle

# 문장을 숫자형태로 바꿔줌
from sklearn.feature_extraction.text import TfidfVectorizer 

# 훈련과 테스트 셋을 나눠줌
from sklearn.model_selection import train_test_split

# 여러개의 종속변수의 라벨데이터를 나이즈베이지안을 해줌
from sklearn.naive_bayes import MultinomialNB

# 문자를 숫자형태로 바꿔서, 가중치를 더한것을 묶어주는것
from sklearn.pipeline import Pipeline

# 결과데이터의 출력해주는 패키지(맞았다, 틀렸다.를 알려줌)
from sklearn.metrics import classification_report

In [7]:
# 데이터 로드
article_df = pd.read_csv('datas/article.csv')
article_df.tail()

,title,link,content,category
595,"""여객법 개정안 처리 촉구"" 카카오모빌리티 등 7곳 공동성명",https://news.naver.com/main/read.nhn?mode=LSD&...,27일 모빌리티 업체 7곳이 여객자동차운수사업법 개정안 처리를 촉구하는 공동 성명을...,105
596,"택배·배달 노동자 ""코로나19 확산 방지 위한 일관된 대책 필요""",https://news.naver.com/main/read.nhn?mode=LSD&...,배송노동 안전지침 마련·비대면 배송 확대·자가격리 지원 요구(지디넷코리아=권상희 ...,105
597,"카카오게임즈 라이프엠엠오, 75억 규모 투자 유치",https://news.naver.com/main/read.nhn?mode=LSD&...,카카오게임즈(각자 대표 남궁훈·조계현)는 자회사 라이프엠엠오(Life MMO)가 7...,105
598,[이슈] RPG로 돌아온 원조 AOS '카오스 모바일' 정식 출시,https://news.naver.com/main/read.nhn?mode=LSD&...,AOS 장르의 원조라 할 수 있는 카오스 온라인을 활용한 모바일게임이 드디어 출시됐...,105
599,"카카오게임즈 프린세스커넥트, 새 캐릭터 시즈루 업데이트",https://news.naver.com/main/read.nhn?mode=LSD&...,"(지디넷코리아=이도원 기자)카카오게임즈(각자 대표 남궁훈, 조계현)는 27일(목)...",105


In [10]:
# 데이터 전처리 : 데이터셋 분리
X_train, X_test, y_train, y_test = train_test_split(
article_df.content,  article_df.category, test_size=0.1, random_state=1)
len(X_train), len(X_test), len(y_train), len(y_test)

(540, 60, 540, 60)

In [14]:
# 모델 만글기
clf = Pipeline([
    # 텍스트 분석에서 거의 사용, 
    # 많이 나온 컬럼데이터단어는 가중치를 낮춰서 분류를 해줌
    # 반대로 적게 나온 데이터단어는 가중치를 높임
    ('vect', TfidfVectorizer()),
    
    ('clf', MultinomialNB(alpha=0.1)),
])

In [13]:
# 학습
model = clf.fit(X_train.values.astype('str'), y_train)

In [19]:
# 모델 성능 확인
# weighted avg : 0.73임, 데이터가 많아지면 조금 덜 할듯
y_pred = model.predict(X_test)
list(y_test[:5]), list(y_pred[:5])
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         100       0.56      1.00      0.71        10
         101       0.62      0.62      0.62         8
         102       0.56      0.50      0.53        10
         103       0.57      0.57      0.57         7
         104       1.00      0.69      0.81        16
         105       0.86      0.67      0.75         9

    accuracy                           0.68        60
   macro avg       0.69      0.68      0.67        60
weighted avg       0.73      0.68      0.69        60



In [21]:
# 모델 사용하기
cateories = {
    100:'정치',
    101:'경제',
    102:'사회',
    103:'생활/문화',
    104:'세계',
    105:'IT/과학'
}

In [22]:
contents = [
    "코로나 확산 방지에 기여",
    '비트코인 시세 급등',
    '민주당 격렬히 저항',
]

datas ={
    'content':contents,
    'cateory_code':model.predict(contents),
}
df = pd.DataFrame(datas)
df

,content,cateory_code
0,코로나 확산 방지에 기여,103
1,비트코인 시세 급등,103
2,민주당 격렬히 저항,100


In [62]:
# 람다 함수 설명
# df['cateory_code']에서 받아오는것을 data 변수로 받고, 이것을 cateories[data]로 변환하는것
# cateories는 딕셔너리이기때문에, cateories[100]을 하면 정치가 나오게 됨.
# 결론은 apply 함수를 통해 모든 df['cateory_code']를 data로 받고 cateories[data]변환하는것.
print(cateories[100])
df['cateory_code'].apply(lambda data : cateories[data])

정치


0    생활/문화
1    생활/문화
2       정치
Name: cateory_code, dtype: object

In [63]:
# proba 설명
# predic안에 proba라는 확률을 나타내는 함수가있고, 리스트로 넣어줘야함
# contents[0]의 단어가 100번 나올확률 0.16, 101번 나올확률 0.086 등.. 해서 마지막에 다 더한 확률이 0.19암
print(model.predict_proba([contents[0]]))
# 위의 내용에서 0번째 배열에서 max값을 가져오면 확률이 제일 높은애를 가져옴
# 너무길어서 소수점 3자리까지만 보겠음
print(round(max(model.predict_proba([contents[0]])[0]),3))

[[0.16793953 0.0865898  0.16104912 0.31743484 0.0694581  0.19752861]]
0.317


In [64]:
# 람다 함수 설명
# df['cateory_code']에서 받아오는것을 data 변수로 받고, 이것을 cateories[data]로 변환하는것
# cateories는 딕셔너리이기때문에, cateories[100]을 하면 정치가 나오게 됨.
# 결론은 apply 함수를 통해 모든 df['cateory_code']를 data로 받고 cateories[data]변환하는것.
# 이것을 df['category']의 새로운 컬럼으로 저장
df['category'] = df['cateory_code'].apply(lambda data : cateories[data])

# proba 설명
# predic안에 proba라는 확률을 나타내는 함수가있고, 리스트로 넣어줘야함
# data의 단어가 100번 나올확률이 있음
# 위의 내용에서 0번째 배열에서 max값을 가져오면 확률이 제일 높은애를 가져옴
# 너무길어서 소수점 3자리까지만 보겠음
df['proba'] = df['content'].apply(lambda data : round(max(model.predict_proba([data])[0]),3))
df

,content,cateory_code,category,proba
0,코로나 확산 방지에 기여,103,생활/문화,0.317
1,비트코인 시세 급등,103,생활/문화,0.215
2,민주당 격렬히 저항,100,정치,0.408


In [66]:
# 모델 저장
pickle.dump(model, open('clf.pkl','wb'))